# Projections and uncertainties of future winter windstorm damage in Europe 
## Notebook 1: Data Processing

This notebook loads, preprocesses, and bias corrects the data required to reproduce the results from the publication Projections and uncertainties of future winter windstorm damage in Europe. Data is taken from the ETH data archive which is linked to the ESGF catalogue, but is not available to the public. However the code can be adapted to any other data archive containing CMIP6 files under netcdf format. Parts of the code has been taken and adapted from the package CASanalysis (https://github.com/islasimpson/CASanalysis) to read the netcdf, and from https://github.com/samluethi/HeatMortality2021/blob/main/Data_preparation/bias_correction.py for the bias-correction.

In [1]:
#load libraries
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import datetime as dt
from numpy import nan
from constants import *
import sys
import warnings
import math
import os
import cftime
from glob import glob
from timeit import default_timer as timer # try to measure time


In [14]:
#setup folders and folder variables.
# All these variables need to be defined correctly for the code to work.
# The strings in these variables should point to existing folders on your computer.
project_folder = '/home/lseverino/MT/scripts MTP'
data_folder = '/home/lseverino/MT/scripts MTP/data/' #this folder will contain the netcdf files downloaded by this script
results_folder = '/home/lseverino/MT/scripts MTP/results2/' #this folder will contain results (e.g. climada impact data)
                                                            # produced by this script
file_identifier = '_v01' # this string is added to all files written by this code

## Check available data in the ETH data archive

In [3]:
## constants
#paths from cmip6 data for each experiments
histpath="/net/atmos/data/cmip6/historical/"
ssp119path="/net/atmos/data/cmip6/ssp119/"
ssp126path="/net/atmos/data/cmip6/ssp126/"
ssp245path="/net/atmos/data/cmip6/ssp245/"
ssp370path="/net/atmos/data/cmip6/ssp370/"
ssp585path="/net/atmos/data/cmip6/ssp585/"

scenlist = ["historical","ssp126","ssp245","ssp370","ssp585"]
pathlist = [histpath,ssp126path,ssp245path,ssp370path,ssp585path]
pathdic = {"historical":histpath,"ssp126":ssp126path,"ssp245":ssp245path,"ssp370":ssp370path,"ssp585":ssp585path}


In [4]:
## check available data
#select data
tres = "day/" #daily data
var="sfcWindmax" #select surface wind max variable

# iterate within each subdir and get names and numbers of available members
dicscen = dict()
models_df = pd.DataFrame(columns=scenlist)
for ind,scen in enumerate(scenlist):
    path = pathlist[ind]
    dicscen[scen] = dict()
    for subdir in os.scandir(path+tres+var):
        #models_rcp85[subdir.name]=[]
        dicscen[scen][subdir.name] = [len(os.listdir(subdir))]
        dicscen[scen][subdir.name].append(os.listdir(subdir))
        models_df.loc[subdir.name,scenlist[ind]] = len(os.listdir(subdir))
models_df.loc["total",:] = models_df.count(axis=0)
nmems_hist = dicscen['historical']
nmems_ssp585 = dicscen['ssp585']
#turn dicts into pd DataFrames
nmems_hist_df = pd.DataFrame(nmems_hist, index=["hist","memnames"])
nmems_ssp585_df = pd.DataFrame(nmems_ssp585, index=["ssp585","memnames"])

In [6]:
models_df

,historical,ssp126,ssp245,ssp370,ssp585
ACCESS-CM2,2,2,2,2,2
ACCESS-ESM1-5,40,40,40,40,40
AWI-CM-1-1-MR,5,1,1,5,1
AWI-ESM-1-1-LR,1,NaN,NaN,NaN,NaN
BCC-CSM2-MR,1,1,1,1,1
BCC-ESM1,1,NaN,NaN,3,NaN
CMCC-CM2-HR4,1,NaN,NaN,NaN,NaN
CMCC-CM2-SR5,11,1,1,1,1
CMCC-ESM2,1,1,1,1,1
CNRM-CM6-1-HR,1,1,1,1,1


## Load daily sfcWindmax data

In [7]:
## select variables
#selvar = 'sfcWindmax'
var = 'sfcWindmax'
tres = 'day'

#spatial domain, in degrees
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

#select scenarios
selscen = ['historical','ssp126','ssp245','ssp370','ssp585']

#max number of members to download
nmems_max = 3

#members to consider: r=realization, i=initialization, p=physics, f=forcing
rlist = [1,2,3] #pick first 3 realizations by default
memnamelist = ["r"+str(ri)+"i1p1f1" for ri in rlist] # use i=1, p=1, and f=1 as default

## choose base name to save data
bnSWM = 'SWM_br_day_EU_winE'

In [8]:
## select dates
#consider ONJDFM: start in D, finishes in M, adjust to have same number of days in both periods
ybegp = 1980 ; monbegp = 10 ; yendp = 2010 ; monendp = 3 ; daybegp = 1 ; dayendp = 30# dates for Past period, only takes 30th
#ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 ; daybegf = 1 ; dayendf = 31# dates for Future period
ybegf = 2070 ; monbegf = 10 ; yendf = 2100 ; monendf = 3 ; daybegf = 1 ; dayendf = 30# otherwise dont have the same length

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
dateformat ='%Y-%m-%d'

datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)+"-"+str(daybegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)+"-"+str(dayendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)+"-"+str(daybegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)+"-"+str(dayendf).zfill(2)

#set up daterange indexes
daysidp = xr.cftime_range(datebegp,dateendp,freq='D',calendar='standard')
daysidf = xr.cftime_range(datebegf,dateendf,freq='D',calendar='standard')

#nb of days
ndaysp = len(daysidp)
ndaysf = len(daysidf)

dayrangep = np.arange(1,ndaysp+1,1)
dayrangef = np.arange(1,ndaysf+1,1)

In [11]:
## select models
#take models that have at least 3 members per scenario
modlist = ['CanESM5','IPSL-CM6A-LR']

#create pandas dataframe to save memnames
iterrows = [modlist_allscen+modlist_ssp585,[i for i in range(nmems_max)]]
rowid = pd.MultiIndex.from_product(iterrows,names=["model","member"])
memname_df = pd.DataFrame(index=rowid, columns=scenlist)
memnames_fn = "memnames_SWM.csv" #savename for csv

#or load already saved csv
#memname_df = pd.read_csv(data_folder+memnames_fn,header=[0],index_col=[0,1])


In [16]:
##open load and process ncdf

for index, modname in enumerate(modlist):
    start_time = timer()
    #select nb of members to consider
    if modname in modlist_allscen:
        nmems = nmems_max #take nmems_max members at most
    else:
        nmems_av = models_df.loc[modname,['historical','ssp585']].min() #only consider members when available for both hist and ssp585
        nmems = min(nmems_av,nmems_max) #take all members if below nmems_max, and nmems_max otherwise


    for scen in selscen:
            #select path
            scenpath = pathdic[scen]

            #get nb of available members and their names
            memdic = dicscen[scen][modname]
            memnb = memdic[0]
            memnames = memdic[1]
            memout=np.arange(0,nmems)
            mem_av = 0

            #pick the correct dates
            if scen == 'historical':
                datebeg = datebegp
                dateend = dateendp
            else:
                datebeg = datebegf
                dateend = dateendf

            for imem in range(nmems):
                memname = memnamelist[imem]
                if memname in memnames:
                    print("Processing "+scen+" for "+modname+" "+memname+"...")
                    #count member actually available
                    mem_av = mem_av + 1
                    memname_df.loc[(modname,imem),scen] = memname
                else:
                    print('Member: ' + memname + 'not available for model: '+modname+' and scen '+ scen)
                    continue

                scendir = glob(scenpath+tres+"/"+var+"/"+modname+"/"+memname+"/*/")
                scendir = scendir[0]

                #read data
                u=read_sfc(scendir+"*.nc", datebeg,dateend)
                uread = u[var]

                # get days index as models can have different calendars
                daysid = uread.time
                ndays = len(daysid)

                #get grid spec (only if first member and keep same specs for following members)
                if imem==0:

                    #use source horizontal resolution of the first member, assuming all members have the same resolution
                    latout = uread.lat
                    lonout = uread.lon

                    #initialize empty array
                    uzmem = xr.DataArray(np.zeros([ ndays, latout.size, lonout.size, nmems]),
                                          coords=[daysid, latout, lonout, memout],dims=['time','lat','lon','member'], name=scen)

                #write into da
                uread = uread.rename({'time': 'time_o'}) #rename to avoid conflicts between indexing and indexed objects

                try:
                    uzmem.loc[dict(member=imem)]=uread
                except IndexError:#if index error, interpolate on the grid of the target data array
                    lati = uzmem.lat
                    loni = uzmem.lon
                    uread = uread.interp(lat=latout,lon=lonout, method='linear',kwargs={"fill_value": None})
                    uzmem.loc[dict(member=imem)]=uread



                #del objects to speed up things?
                del u
                del uread

            #skip rest if no members were found
            if mem_av==0:
                continue


            ## processings
            # normalize longitude
            uzmemn = norm_lon(uzmem)

            #interpolate nans at 0deg longitude
            if np.any(np.isnan(uzmemn.dropna(dim="time",how='all'))):
                print('interpolating nans...')
                uzmemn = uzmemn.interpolate_na(dim="lon", method="linear")

            #crop to EU domain
            Usfc_EU = def_domain(uzmemn,min_lat,max_lat,min_lon,max_lon)

            #set time indexes
            dayrange = np.arange(1,ndays+1,1)
            Usfc_EU = Usfc_EU.assign_coords({"day":("time",dayrange)})

            #select winter months
            Usfc_EU_win = get_ONDJFM_day(Usfc_EU,timedim="time")


            #swap dims to assemble files in the same dataset

            Usfc_EU_win = Usfc_EU_win.swap_dims({"time":"day"})

            #set time index as a data variable
            Usfc_EU_win = Usfc_EU_win.reset_coords()

            if scen == 'historical':
                timename = 'timep'
            else:
                timename = 'timef'
            Usfc_EU_win = Usfc_EU_win.rename(time=timename)

            #save to netcdf

            try:# see if file exist and merge to it

                Usfc_EU_win.to_netcdf(path=data_folder+modname+'_'+bnSWM+".nc",mode="a")
            except: #otherwise directly create the file
                Usfc_EU_win.to_netcdf(path=data_folder+modname+'_'+bnSWM+".nc")


            time_delta_fut = timer() - start_time
            print(time_delta_fut)


memname_df.to_csv(data_folder+memnames_fn)

Processing historical for CanESM5 r1i1p1f1...
Processing historical for CanESM5 r2i1p1f1...
Processing historical for CanESM5 r3i1p1f1...
56.78150917496532
Processing ssp126 for CanESM5 r1i1p1f1...
Processing ssp126 for CanESM5 r2i1p1f1...
Processing ssp126 for CanESM5 r3i1p1f1...
180.84883580496535
Processing ssp245 for CanESM5 r1i1p1f1...
Processing ssp245 for CanESM5 r2i1p1f1...
Processing ssp245 for CanESM5 r3i1p1f1...
275.05035652406514
Processing ssp370 for CanESM5 r1i1p1f1...
Processing ssp370 for CanESM5 r2i1p1f1...
Processing ssp370 for CanESM5 r3i1p1f1...
367.2344959098846
Processing ssp585 for CanESM5 r1i1p1f1...
Processing ssp585 for CanESM5 r2i1p1f1...
Processing ssp585 for CanESM5 r3i1p1f1...
481.6006460310891
Processing historical for IPSL-CM6A-LR r1i1p1f1...
Processing historical for IPSL-CM6A-LR r2i1p1f1...
Processing historical for IPSL-CM6A-LR r3i1p1f1...
151.94626605371013
Processing ssp126 for IPSL-CM6A-LR r1i1p1f1...
Processing ssp126 for IPSL-CM6A-LR r2i1p1f1...


## Load U850

In [17]:
#check available data
tres = "Amon/" #take monthly data
var="ua" #select zonal wind

dicscen = dict()
models_df = pd.DataFrame(columns=scenlist)
for ind,scen in enumerate(scenlist):
    path = pathlist[ind]
    dicscen[scen] = dict()
    for subdir in os.scandir(path+tres+var):
        #models_rcp85[subdir.name]=[]
        dicscen[scen][subdir.name] = [len(os.listdir(subdir))]
        dicscen[scen][subdir.name].append(os.listdir(subdir))
        models_df.loc[subdir.name,scenlist[ind]] = len(os.listdir(subdir))
models_df.loc["total",:] = models_df.count(axis=0)
nmems_hist = dicscen['historical']
nmems_ssp585 = dicscen['ssp585']

In [23]:
#get member names used for SWM
memname_df = pd.read_csv(data_folder+memnames_fn,header=[0],index_col=[0,1])

#models which have at least 1 member for historical and ssp585
models_UA_1mem = models_df[['historical','ssp585']].where(models_df>=1)

#keep same models used for SWM
models_UA = models_UA_1mem.reindex(modlist_ssp585+modlist_allscen)
modlist_UA = models_UA.index.tolist()
modlist = ['CanESM5','IPSL-CM6A-LR']


In [25]:
## select variables
#selvar = 'sfcWindmax'
var = 'ua'
tres = 'Amon'

#spatial domain
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

plev = [85000] #pressure level

#members
nmems_max = 1 #max number of members to consider

#select scenarios
selscen = ['historical','ssp585']

#normalization to a regular grid
norm=False

#names to save files
bnU850 = 'U850_br_day_EU_winE'

In [26]:
##open load and process ncdf
nmems = 1
memname_dl_UA = pd.DataFrame(index=modlist_UA)

for index, modname in enumerate(modlist):
    start_time = timer()

    for scen in selscen:
            #select path
            scenpath = pathdic[scen]
            #get nb of available members and their names
            memdic = dicscen[scen][modname]
            memnb = memdic[0]
            memnames = memdic[1]
            memout=np.arange(0,nmems)
            mem_av = 0
            #select correct dates
            if scen == 'historical':
                datebeg = datebegp
                dateend = dateendp
            else:
                datebeg = datebegf
                dateend = dateendf

            for imem in range(nmems):

                memname = memname_df.loc[(modname,0),scen] #pick first members used for SWM data

                if memname in memnames:
                    print("Processing "+scen+" for "+modname+" "+memname+"...")
                    #count member actually available
                    mem_av = mem_av + 1
                    memname_dl_UA.loc[modname,scen] = memname
                else:
                    print('Member: ' + memname + 'not available for model: '+modname+' and scen '+ scen)
                    continue


                scendir = glob(scenpath+tres+"/"+var+"/"+modname+"/"+memname+"/*/")
                scendir = scendir[0]
                try:
                    field = read_field(scendir+"*.nc", datebeg,dateend,min_lat,max_lat,None,None,plev)

                except ValueError:
                    #assume first file contains what we need
                    fpath = glob(scenpath+tres+var+"/"+modname+"/"+memname+"/*//*.nc")[0]
                    print("Look into: "+fpath)
                    field = read_field(fpath,datebeg,dateend,min_lat,max_lat,None,None,plev)

                utemp = field[var]

                #get days, necessayry as models can have different calendars
                daysid = utemp.time
                ndays = len(daysid)

                #take mean over pressure
                utemp = utemp.mean(dim='plev')

                #get grid spec (only if first member and keep same specs for following members)
                if imem==0:

                    #use source resolution
                    latout = utemp.lat
                    lonout = utemp.lon

                    #initialize empty array
                    uzmem = xr.DataArray(np.zeros([ ndays, latout.size, lonout.size, nmems]),
                                          coords=[daysid, latout, lonout, memout],dims=['time','lat','lon','member'], name=scen)

                #read data
                if norm: #interpolation if regridding/regularization
                    uread = utemp.interp(lat=latout,lon=lonout, method='linear',kwargs={"fill_value": None})

                else: #slicing otherwise
                    uread = utemp
                    #uread = utemp.sel(lat=slice(minlato,maxlato),lon=slice(minlono,maxlono))

                #write into da
                uread = uread.rename({'time': 'time_o'}) #rename to avoid conflicts between indexing and indexed objects

                try:
                    uzmem.loc[dict(member=imem)]=uread
                except IndexError:#if index error, interpolate on the grid of the target data array
                    lati = uzmem.lat
                    loni = uzmem.lon
                    uread = uread.interp(lat=latout,lon=lonout, method='linear',kwargs={"fill_value": None})


                #del objects to speed up things?
                del field
                del utemp
                del uread

            #skip rest if no members were found
            if mem_av==0:
                continue


            ## processings
            # normalize longitude
            uzmemn = norm_lon(uzmem)

            #interpolate nans at 0deg longitude
            if np.any(np.isnan(uzmemn.dropna(dim="time",how='all'))):
                print('interpolating nans...')
                uzmemn = uzmemn.interpolate_na(dim="lon", method="linear")

            #crop to domain
            Usfc_EU = def_domain(uzmemn,min_lat,max_lat,min_lon,max_lon)

            #set time indexes
            dayrange = np.arange(1,ndays+1,1)
            Usfc_EU = Usfc_EU.assign_coords({"day":("time",dayrange)})

            #select winter months
            Usfc_EU_win = get_ONDJFM_day(Usfc_EU,timedim="time")

            #swap dims to assemble files in the same dataset
            Usfc_EU_win = Usfc_EU_win.swap_dims({"time":"day"})

            #set time index as a data variable
            Usfc_EU_win = Usfc_EU_win.reset_coords()

            if scen == 'historical':
                timename = 'timep'
            else:
                timename = 'timef'
            Usfc_EU_win = Usfc_EU_win.rename(time=timename)

            #save to netcdf

            try:# see if file exist and merge to it
                Usfc_EU_win.to_netcdf(path=data_folder+modname+'_'+bnU850+'.nc',mode="a")
            except: #otherwise directly create the file
                Usfc_EU_win.to_netcdf(path=data_folder+modname+'_'+bnU850+'.nc')


            time_delta_fut = timer() - start_time
            print(time_delta_fut)

Processing historical for CanESM5 r1i1p1f1...
14.081292682792991
Processing ssp585 for CanESM5 r1i1p1f1...
21.117706035729498
Processing historical for IPSL-CM6A-LR r1i1p1f1...
interpolating nans...
21.89428939903155
Processing ssp585 for IPSL-CM6A-LR r1i1p1f1...
interpolating nans...
36.13658937579021


## Load ERA5 WG10 data

In [1]:
import dask

In [45]:
## constants
#paths
pathin = "/net/atmos/data/era5" #input path for ERA5 data
#time index
ybegp = 1980 ; monbegp = 1 ; yendp = 2010 ; monendp = 12 ; daybegp = 1 ; dayendp = 31# dates for Past period, only takes 30th

years = np.arange(ybegp,yendp+1)
months = ["01","02","03","10","11","12"]
#lat lon
latout=np.linspace(-90,90,73)
lonout=np.linspace(0,357.5,144)
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

selvars = ['U10M','V10M']
data_vars = ['PS','MSL','TCC','U10M','V10M','SSTK','CI','D2M','T2M','TCW','TCWV','VIWVD','E',
             'MN2T','MX2T','SI','TTR','TTRC','WG10','LSP','CP','SF','SSHF','SLHF','BLH']
to_drop = ['PS','MSL','TCC','SSTK','CI','D2M','T2M','TCW','TCWV','VIWVD','E',
             'MN2T','MX2T','SI','TTR','TTRC','WG10','LSP','CP','SF','SSHF','SLHF','BLH']


In [46]:
#get pattern
pathlist = []
for yr in years:
    for mn in months:
        pathinvar = pathin+"/"+str(yr)+"/"+mn+'/B'+str(yr)+mn+'??_??'
        pathlist+=glob(pathinvar)


In [58]:
# preprocess
def def_domain(ds,min_lat=30,max_lat=75,min_lon=-30,max_lon=30):
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds.sortby(ds.lon)
    LatIndexer, LonIndexer = 'lat', 'lon'
    ds = ds.loc[{LatIndexer: slice(min_lat, max_lat),
                      LonIndexer: slice(min_lon, max_lon)}]
    return ds
def get_ONDJFM_day(ds, months=[1,2,3,10,11,12]):
    if "time" not in ds.dims:
        ds = ds.swap_dims({"day":"time"})
    return ds.isel(time=ds.time.dt.month.isin(months))

def norm_wind(ds):
    u = ds['U10M']
    v = ds['V10M']
    return np.sqrt(u**2+v**2)

def daily_max(ds):
    return ds.resample(time="1D").max()

def preprocess(ds,**kwargs):
    def def_domain(ds,min_lat=30,max_lat=75,min_lon=-30,max_lon=30):
        ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
        ds.sortby(ds.lon)
        LatIndexer, LonIndexer = 'lat', 'lon'
        ds = ds.loc[{LatIndexer: slice(min_lat, max_lat),
                          LonIndexer: slice(min_lon, max_lon)}]
        return ds
    #drop useless var
    ds = ds['WG10']
    #slice domain
    ds = def_domain(ds)

    return ds


In [48]:
#open data set using mfdataset and preprocess
start_time = timer()

ds = xr.open_mfdataset(pathlist, concat_dim="time", combine="nested",
                  data_vars='all', coords='minimal', compat='override',preprocess=preprocess,parallel=True)

time_delta_fut = timer() - start_time
print(time_delta_fut)

24167.72115666396


In [49]:
#resample to daily maxima
start_time = timer()
ds_sort = ds.sortby(ds.time)
ds_day = ds_sort.resample(time="1D").max()
time_delta_fut = timer() - start_time
print(time_delta_fut)

10824.226235975977


In [59]:
#get winter days
start_time = timer()
ds_day_winE = get_ONDJFM_day(ds_day, timedim="time")
time_delta_fut = timer() - start_time
print(time_delta_fut)

0.33201643207576126


In [13]:
#save data under netcdf format
fnera5 = 'era5_WG10_br_day_EU_winE.nc' #savename
ds_day_winE.to_netcdf(path=data_folder+fnera5)

NameError: name 'ds_day_winE' is not defined

# Bias correction


In [27]:
#import bias correction module
from SL_bias_corr import *

In [28]:
#select climate models, used scenarios (historical, ssp585,...)
modlist =  ['CanESM5','IPSL-CM6A-LR']
scen_used= ['historical','ssp585']
pastname = 'historical'

bncmip6 = bnSWM #filename of the data to be bias-corrected
fnera5 = 'era5_WG10_br_day_EU_winE.nc' #filename of the control data for bias-correction
bnout = "bias_corrWG10_"+bncmip6 #filename of the output of the bias correction


In [29]:
##Bias correction
#Loop over, scenarios, models and model members
#open netcdf and do preprocessing
#Use stack=True to stack model members on top of each other, set stack=False to consider each member separately.
#Use savehaz, saveimpcsv, saveimpmat to save hazards or/and imapcts.

##parameters
savencdf = True

#load era5 data
with xr.open_dataset(data_folder+fnera5) as era_ds:
    era_da = era_ds['WG10']

for modid, modname in enumerate(modlist):

    #read netcdf
    fncmip6 = make_fn([modname],bncmip6,filetype=".nc")

    with xr.open_dataset(data_folder+fncmip6) as cmip_ds:
        if modname == 'NESM3': #drop member r2i1p1f1 of NESM3 because faulty
            cmip_ds = cmip_ds.drop_sel(member=1)
        #get coordinates
        latout = cmip_ds.lat
        lonout = cmip_ds.lon
        daysid = cmip_ds.day
        memid = cmip_ds.member

        #select past and future period and get correct time coord
        past_da = cmip_ds.swap_dims({"day":"timep"}).reset_coords()[pastname]

        fut_da = cmip_ds.swap_dims({"day":"timef"}).reset_coords().drop_vars(["historical","timep","day"])

    #convert calendars
    past_da = past_da.convert_calendar('standard', dim='timep',align_on="year")
    past_tid = past_da.timep

    fut_da = fut_da.convert_calendar('standard', dim='timef',align_on="year")
    fut_tid = fut_da.timef

    #regrid era5 to the gcm's res
    era_qt_rg = era_da.interp(lat=latout,lon=lonout, method='linear',kwargs={"fill_value": 'extrapolate'}) #regridded era5

    #initiate dict to store data
    da_dict = dict()

    for scen in scen_used:
        start_time = timer() #start counting time
        last_time = start_time

        ncdfw = cmip_ds[[pastname,scen]] #select scen

        #initialize empty array
        da_scen = xr.DataArray(np.zeros([ daysid.size, latout.size, lonout.size, memid.size]),
                          coords=[daysid, latout, lonout, memid],dims=['time','lat','lon','member'], name=scen)

        ##bias correction

        #iterate over lat and lon
        for ilat in latout:
            for ilon in lonout:
                ncdfw_df = ncdfw.sel(lat=ilat,lon=ilon).to_dataframe().drop(['lat','lon'],axis=1).unstack()

                dat_mod = ncdfw_df[pastname].copy()
                dat_mod_all =  ncdfw_df[scen].copy()

                #correct time index
                dat_mod.index = past_tid
                dat_mod_all.index = fut_tid

                #keep only date format
                dat_mod.index = dat_mod.index.date
                dat_mod_all.index = dat_mod_all.index.date

                #select obs gridpoint
                dat_obs =  era_qt_rg.sel(lat=ilat,lon=ilon).to_dataframe().drop(['lat','lon'],axis=1)['WG10']

                #reindex GCM time data on obs time data
                dat_mod = dat_mod.reindex(dat_obs.index.date).dropna()

                dat_mod_all_corrected = bias_correct_ensemble(dat_mod, dat_obs, dat_mod_all, minq=0.001, maxq=1.000, incq=0.001)

                da_scen.loc[dict(lat=ilat,lon=ilon)] = dat_mod_all_corrected

        da_dict[scen] = da_scen
        timef = timer()
        time_delta_past3 = timef - start_time
        print("Time for bias correction model "+modname+" and scen "+scen+": "+str(time_delta_past3))


    #create dataset
    bias_corr_ds = xr.Dataset(da_dict)

    #keep correct time coordinates as variables
    bias_corr_ds = bias_corr_ds.rename(time="day")
    bias_corr_ds = bias_corr_ds.assign_coords({"timep":("day",past_tid.data)}).reset_coords()
    bias_corr_ds = bias_corr_ds.assign_coords({"timef":("day",fut_tid.data)}).reset_coords()

    ##save files
    if savencdf:
        bias_corr_ds.to_netcdf(path=data_folder+modname+"_"+bnout+'.nc')


Time for bias correction model CanESM5 and scen historical: 8.866932092234492
Time for bias correction model CanESM5 and scen ssp585: 11.37633218523115
Time for bias correction model IPSL-CM6A-LR and scen historical: 30.782586419954896
Time for bias correction model IPSL-CM6A-LR and scen ssp585: 53.61053590895608
